В этом ноутбуке обучим модель RoBERT aраспознавать спам в SMS

In [1]:
model_name = 'roberta'
train_dataset_name = 'spam sms'

In [2]:
MAX_LEN = 128 

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

In [5]:
!pip install tensorflow-text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Загружаем данные

In [7]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/data_for_colab/spam_sms.csv', encoding = "ISO-8859-1")
df

Mounted at /content/drive


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [8]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
df.columns = ['IS_SPAM', 'DATA_COLUMN']
df['IS_SPAM'] = (df['IS_SPAM'] == 'spam').astype(int)

In [102]:
parameter_const = 1.8 # подобрали
df['DATA_COLUMN'] = df['DATA_COLUMN'].apply(lambda x: x[:round(MAX_LEN * parameter_const)])

In [103]:
df

,IS_SPAM,DATA_COLUMN
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [104]:
df['IS_SPAM'].value_counts()

0    4825
1     747
Name: IS_SPAM, dtype: int64

In [105]:
df_positive = df[df['IS_SPAM']==1]

In [106]:
df_negative = df[df['IS_SPAM']==0]

Создаем тестовую и обучающую выборки

In [107]:
# Тестовая выборка
n_test = df_negative.shape[0] // 2
df_negative_test = df_negative.tail(n_test)
n_test = df_positive.shape[0] // 2
df_positive_test = df_positive.tail(n_test)

In [108]:
df_negative_test.shape

(2412, 2)

In [109]:
df_positive_test.shape

(373, 2)

In [110]:
df_positive_test

,IS_SPAM,DATA_COLUMN
2728,1,Urgent Please call 09066612661 from landline. ...
2729,1,Urgent! Please call 09066612661 from your land...
2741,1,I don't know u and u don't know me. Send CHAT ...
2766,1,Married local women looking for discreet actio...
2769,1,Burger King - Wanna play footy at a top stadiu...
...,...,...
5537,1,Want explicit SEX in 30 secs? Ring 02073162414...
5540,1,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,1,Had your contract mobile 11 Mnths? Latest Moto...
5566,1,REMINDER FROM O2: To get 2.50 pounds free call...


In [111]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [112]:
df_balanced_test.sample(10)

,IS_SPAM,DATA_COLUMN
3936,0,"Yeah, in fact he just asked if we needed anyth..."
4803,0,"Er, hello, things didnÛ÷t quite go to plan Û..."
4845,0,Pls help me tell Ashley that i cant find her n...
3136,0,You're right I have now that I think about it
5376,0,I don't want you to leave. But i'm barely doin...
5476,0,Yes princess! I want to please you every night...
3283,0,ALRITE SAM ITS NIC JUST CHECKIN THAT THIS IS U...
4514,0,That was random saw my old roomate on campus. ...
2827,0,Wife.how she knew the time of murder exactly
2904,0,Ha. You donÛ÷t know either. I did a a clever ...


In [113]:
df_balanced_test['IS_SPAM'].value_counts()

0    2412
1     373
Name: IS_SPAM, dtype: int64

In [114]:
# Обучающая выборка
n_train = df_negative.shape[0] // 2
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 2
df_positive_train = df_positive.head(n_train)

In [115]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [116]:
df_balanced_train['IS_SPAM'].value_counts()

0    2412
1     373
Name: IS_SPAM, dtype: int64

In [117]:
df_balanced_train.sample(10)

,IS_SPAM,DATA_COLUMN
199,0,"Found it, ENC &lt;#&gt; , where you at?"
347,0,One small prestige problem now.
1423,0,Lol great now im getting hungry.
995,0,Change again... It's e one next to escalator...
2308,1,Moby Pub Quiz.Win a å£100 High Street prize if...
1270,0,"Sorry chikku, my cell got some problem thts y ..."
2133,0,Spoke with uncle john today. He strongly feels...
669,0,Did u receive my msg?
2213,0,"Goodmorning, today i am late for 2hrs. Because..."
5,1,FreeMsg Hey there darling it's been 3 week's n...


In [127]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze().values
y_train = df_balanced_train['IS_SPAM'].squeeze().values

In [128]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze().values
y_test = df_balanced_test['IS_SPAM'].squeeze().values

Работаем с моделью

In [129]:
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [130]:
max_len = MAX_LEN
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.BinaryCrossentropy()

input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
output = roberta_model.roberta([input_ids,attention_masks])
output = output[1]
output = tf.keras.layers.Dense(1, activation='sigmoid')(output)
model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer=opt, loss=loss, metrics=METRICS)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 128)]        0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 128)]        0           []                               
                                                                                                  
 roberta (TFRobertaMainLayer)   TFBaseModelOutputWi  124645632   ['input_11[0][0]',               
                                thPoolingAndCrossAt               'input_12[0][0]']               
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [131]:
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")

In [132]:
# код ниже нужен для проверки
token_lens = []

for txt in X_train:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 94


In [133]:
for txt in X_test:
    tokens = tokenizer_roberta.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
max_length=np.max(token_lens)
print('max_length is', max_length)
if max_length >= MAX_LEN:
    raise ValueError("Hyperparameter MAX_LEN must be >= max_len for both train, valid and test data")

max_length is 96


In [134]:
def tokenize_roberta(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [135]:
train_input_ids, train_attention_masks = tokenize_roberta(X_train, MAX_LEN)
test_input_ids, test_attention_masks = tokenize_roberta(X_test, MAX_LEN)

In [136]:
history = model.fit([train_input_ids,train_attention_masks], y_train, epochs=5, batch_size=30)

Epoch 1/5
93/93 [==============================] - 111s 773ms/step - loss: 0.1811 - accuracy: 0.9185 - precision: 0.6995 - recall: 0.6863
Epoch 2/5
93/93 [==============================] - 68s 727ms/step - loss: 0.0176 - accuracy: 0.9964 - precision: 0.9973 - recall: 0.9759
Epoch 3/5
93/93 [==============================] - 67s 717ms/step - loss: 0.0049 - accuracy: 0.9986 - precision: 0.9973 - recall: 0.9920
Epoch 4/5
93/93 [==============================] - 67s 715ms/step - loss: 0.0125 - accuracy: 0.9961 - precision: 0.9866 - recall: 0.9839
Epoch 5/5
93/93 [==============================] - 66s 715ms/step - loss: 0.0034 - accuracy: 0.9993 - precision: 1.0000 - recall: 0.9946


In [137]:
result_roberta = model.predict([test_input_ids,test_attention_masks])

88/88 [==============================] - 28s 267ms/step


In [138]:
y_predicted = np.where(result_roberta > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [139]:
y_test

array([0, 0, 0, ..., 1, 1, 1])

In [140]:
accuracy_score(y_test, y_predicted)

0.99245960502693

In [141]:
precision_score(y_test, y_predicted)


0.9705882352941176

In [142]:
recall_score(y_test, y_predicted)

0.9731903485254692

In [143]:
f1_score(y_test, y_predicted)

0.9718875502008032

In [144]:
df_results_on_test = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [145]:
index = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'
df_results_on_test.loc[index, 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_on_test.loc[index, 'precision'] = precision_score(y_test, y_predicted)
df_results_on_test.loc[index, 'recall'] = recall_score(y_test, y_predicted)
df_results_on_test.loc[index, 'f1_score'] =  f1_score(y_test, y_predicted)

In [146]:
df_results_on_test

,accuracy,precision,recall,f1_score
roberta trained on spam sms and tested on spam sms dataset,0.99246,0.970588,0.97319,0.971888


Сохраним обученную модель

In [147]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/roberta_trained_on_spam_sms_20_january'

In [148]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/roberta_trained_on_spam_sms_20_january


In [149]:
model.save(saved_model_path, include_optimizer=True) 

Сохраним датафреймы с результатами на обучающей выборке и на тестовой выборке

Сначала результаты для обучающей выборки

In [150]:
name_for_train_csv = model_name + ' trained on ' + train_dataset_name + ' quality on train dataset'

In [151]:
def calculate_f1_score(precision, recall):
    F1 = 2 * (precision * recall) / (precision + recall)
    return F1

In [152]:
df_with_train_quality = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [153]:
for el in ['accuracy', 'precision', 'recall']:
    df_with_train_quality.loc[name_for_train_csv, el] = history.history[el][-1]
df_with_train_quality.loc[name_for_train_csv, 'f1_score'] = calculate_f1_score(history.history['precision'][-1], history.history['recall'][-1])

In [154]:
df_with_train_quality

,accuracy,precision,recall,f1_score
roberta trained on spam sms quality on train dataset,0.999282,1.0,0.994638,0.997312


In [155]:
df_with_train_quality.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/train_quality/' + name_for_train_csv + '.csv')

Теперь результаты для тестовой выборки

In [156]:
name_for_test_csv = model_name + ' trained on ' + train_dataset_name + ' and tested on ' + train_dataset_name + ' dataset'

In [157]:
df_results_on_test.to_csv('/content/drive/MyDrive/data_for_colab/dataframes/test_quality/' + name_for_test_csv + '.csv')